In [1]:
import os
import cv2
import random
import math
import numpy as np
import datetime as dt
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/Users/vedantpadole/Desktop/Research/ASL_MY_DATASET')

all_classes = os.listdir('/Users/vedantpadole/Desktop/Research/ASL_MY_DATASET')
print (all_classes)  
len(all_classes)

['sugar', 'cup', 'sweet', 'ac', 'bill', 'thank-you', 'water', 'no', 'vegetables', 'pop corn', 'bitter', 'milk', 'allergy', 'spicy', 'cost', 'ice cream', 'chair', 'cheese', 'napkin', 'about', 'small', 'french fries', 'manager', 'salt', 'cold', 'warm', 'ingredients', 'burger', 'alcohol', 'eggs', 'chicken', 'what', 'bread', 'hello', 'sauce', 'bag', 'pizza', 'pepper', 'drink', 'which', 'gluten free']


41

In [3]:
plt.figure(figsize=(20,20))
random_range=random.sample(range(len(all_classes)),22)
for counter,random_index in enumerate(random_range,1):
  selected_Class=all_classes[random_index]
  video_files_names_list=os.listdir(f'/Users/vedantpadole/Desktop/Research/ASL_MY_DATASET/{selected_Class}')
  selected_video_file_name=random.choice(video_files_names_list)
  video_reader=cv2.VideoCapture(f'/Users/vedantpadole/Desktop/Research/ASL_MY_DATASET/{selected_Class}/{selected_video_file_name}')
  _,bgr_frame=video_reader.read()
  rgb_frame=cv2.cvtColor(bgr_frame,cv2.COLOR_BGR2RGB)
  cv2.putText(rgb_frame,selected_Class,(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
  plt.subplot(11,2,counter);plt.imshow(rgb_frame);plt.axis('off')

In [4]:
IMAGE_HEIGHT,IMAGE_WIDTH=128,128
SEQUENCE_LENGTH=10
DATASET='/Users/vedantpadole/Desktop/Research/ASL_MY_DATASET'
CLASSES_LIST=all_classes

In [5]:
def frames_extraction(video_path):
  frames_list=[]
  video_reader=cv2.VideoCapture(video_path)
  video_frames_count=int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
  skip_frames_window=max(int(video_frames_count/SEQUENCE_LENGTH),1)
  for frame_counter in range(SEQUENCE_LENGTH):
    video_reader.set(cv2.CAP_PROP_POS_FRAMES,frame_counter*skip_frames_window)
    success,frame=video_reader.read()
    if not success:
      break
    resized_frame=cv2.resize(frame,(IMAGE_HEIGHT,IMAGE_WIDTH))
    normalized_frame=resized_frame/255
    frames_list.append(normalized_frame)
  video_reader.release()
  return frames_list

In [6]:
def create_dataset():
  features=[]
  labels=[]
  video_files_path=[]
  for class_index,class_name in enumerate(CLASSES_LIST):
    print(f'Extracting the data of class:  {class_name}')
    files_list=os.listdir(os.path.join(DATASET,class_name))
    for file_name in files_list:
      video_file_path=os.path.join(DATASET,class_name,file_name)
      frames=frames_extraction(video_file_path)
      if len(frames)==SEQUENCE_LENGTH:
        features.append(frames)
        labels.append(class_index)
        video_files_path.append(video_file_path)
  features=np.asarray(features)
  labels=np.array(labels)
  return features,labels,video_files_path

In [7]:
features,labels,video_files_path=create_dataset()

Extracting the data of class:  sugar
Extracting the data of class:  cup
Extracting the data of class:  sweet
Extracting the data of class:  ac
Extracting the data of class:  bill
Extracting the data of class:  thank-you
Extracting the data of class:  water
Extracting the data of class:  no
Extracting the data of class:  vegetables
Extracting the data of class:  pop corn
Extracting the data of class:  bitter
Extracting the data of class:  milk
Extracting the data of class:  allergy
Extracting the data of class:  spicy
Extracting the data of class:  cost
Extracting the data of class:  ice cream
Extracting the data of class:  chair
Extracting the data of class:  cheese
Extracting the data of class:  napkin
Extracting the data of class:  about
Extracting the data of class:  small
Extracting the data of class:  french fries
Extracting the data of class:  manager
Extracting the data of class:  salt
Extracting the data of class:  cold
Extracting the data of class:  warm
Extracting the data of

In [8]:
from keras.utils import to_categorical
one_hot_encoded_labels=to_categorical(labels)

In [9]:
seed_constant=27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [10]:
from sklearn.model_selection import train_test_split
features_train,features_test,labels_train,labels_test=train_test_split(features,one_hot_encoded_labels,test_size=0.2,shuffle=True,random_state=seed_constant)

In [11]:
a=features_train.shape[0]

In [12]:
from keras import Sequential
from keras.layers import ConvLSTM2D, MaxPooling3D, TimeDistributed, Dropout, Flatten, Dense, Bidirectional, LSTM

def create_bidirectional_convlstm_model():
    model = Sequential()
    model.add(ConvLSTM2D(filters=4, kernel_size=(3, 3), activation='relu', data_format='channels_last',
                         recurrent_dropout=0.2, return_sequences=True, input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters=14, kernel_size=(3, 3), activation='relu', data_format='channels_last',
                         recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters=16, kernel_size=(3, 3), activation='relu', data_format='channels_last',
                         recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(TimeDistributed(Flatten()))
    model.add(Bidirectional(LSTM(units=32, activation='relu', return_sequences=True)))
    model.add(Bidirectional(LSTM(units=64, activation='relu', return_sequences=True)))
    model.add(Flatten())
    model.add(Dense(len(CLASSES_LIST), activation='softmax'))
    model.summary()
    return model


In [13]:
model1=create_bidirectional_convlstm_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 10, 126, 126, 4)   1024      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 10, 63, 63, 4)    0         
 )                                                               
                                                                 
 time_distributed (TimeDistr  (None, 10, 63, 63, 4)    0         
 ibuted)                                                         
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 10, 61, 61, 14)    9128      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 10, 31, 31, 14)   0         
 3D)                                                             
                                                        

In [14]:
from keras.utils import plot_model
plot_model(model1,to_file='model1.png',show_shapes=True,show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [15]:
from keras.callbacks import EarlyStopping
early_stopping_call=EarlyStopping(monitor='val_loss',patience=10,mode='min',restore_best_weights=True)
model1.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
r=model1.fit(x=features_train,y=labels_train,epochs=50,batch_size=32,shuffle=True,validation_split=0.2,callbacks=[early_stopping_call])

Epoch 1/50
25/25 [==============================] - 480s 18s/step - loss: 3.6650 - accuracy: 0.0368 - val_loss: 3.6666 - val_accuracy: 0.0354
Epoch 2/50
25/25 [==============================] - 458s 18s/step - loss: 3.4500 - accuracy: 0.0824 - val_loss: 3.4545 - val_accuracy: 0.0657
Epoch 3/50
25/25 [==============================] - 455s 18s/step - loss: 3.2882 - accuracy: 0.1103 - val_loss: 3.3082 - val_accuracy: 0.0859
Epoch 4/50
25/25 [==============================] - 463s 19s/step - loss: 2.8763 - accuracy: 0.2167 - val_loss: 3.0809 - val_accuracy: 0.1869
Epoch 5/50
25/25 [==============================] - 507s 20s/step - loss: 2.4771 - accuracy: 0.2991 - val_loss: 2.7046 - val_accuracy: 0.2677
Epoch 6/50
25/25 [==============================] - 512s 20s/step - loss: 2.1245 - accuracy: 0.3587 - val_loss: 2.2618 - val_accuracy: 0.4242
Epoch 7/50
25/25 [==============================] - 521s 21s/step - loss: 1.6846 - accuracy: 0.4956 - val_loss: 1.9151 - val_accuracy: 0.4091
Epoch 

In [ ]:
model1.